In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import pyplot
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle

## Process Data and Feature Extraction

In [2]:
#read in dataset, drop the duplicate rows where a song appears twice for same date with multiple artists
df = pd.read_csv("Spotify_Dataset_V3.csv", delimiter = ";")
df = df.drop_duplicates(subset = ['Title', 'Artists', 'Date'], keep = 'first')
df = df.reset_index(drop = True)

#convert date to datetime and sort in nondescending order
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values(by='Date')

#find a songs highest rank ever in dataset (min value)
df['Highest Rank'] = df.groupby(['Title', 'Artists'])['Rank'].transform('min')

In [3]:
#create new columns number of songs previous and previous popularity. 
#Track if an artist is ever considered top ten (i.e. any of an artist's song rank reached <= 10)
#Track the number of songs an artist previously has, since we don't have an artist id, we will assume the artist name is unique
#At each row, update the new column based on the data we have to that point. This way we do not use future information to
#explain past rank
artist_top_ten = {}
artist_songs_previous = {}
length_in_charts = {}
for index, row in df.iterrows():
    artist = row['Artists']
    song = row["Title"]
    if artist not in artist_top_ten:
        artist_top_ten[artist] = False
    if row["Rank"] <= 10 and (song in artist_songs_previous.get(artist, [])):
        artist_top_ten[artist] = True
    if artist_top_ten[artist] == True: 
        df.at[index, "Previous Popularity"] = 1
    else:
        df.at[index, "Previous Popularity"] = 0
        
    
    if artist in artist_songs_previous:
        if song not in artist_songs_previous[artist]:
            number = len(artist_songs_previous[artist])
            df.at[index, "Number Songs Previous"] = number 
            artist_songs_previous[artist].append(song)
        else:
            number = len(artist_songs_previous[artist])
            df.at[index, "Number Songs Previous"] = number
    else:
            # If artist is not in the dictionary, create a new entry for the artist
            artist_songs_previous[artist] = [song]
            df.at[index, "Number Songs Previous"] = 0 
          

In [4]:
#print columns of the cleaned dataset
df.columns

Index(['Rank', 'Title', 'Artists', 'Date', 'Danceability', 'Energy',
       'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Valence', '# of Artist', 'Artist (Ind.)', '# of Nationality',
       'Nationality', 'Continent', 'Points (Total)',
       'Points (Ind for each Artist/Nat)', 'id', 'Song URL', 'Highest Rank',
       'Previous Popularity', 'Number Songs Previous'],
      dtype='object')

In [5]:
#number of artists for a song
for index, row in df.iterrows():
    artists = row['Artists'].split(", ")
    df.at[index, "Number of Artists"] = len(artists)

In [6]:
#set top 1 true or false if highest rank is <= 1
df['Top 10'] = df['Highest Rank'].apply(lambda x: True if x <= 10 else False)

#only keep one song in df as grouped by title and artists keeping the first
df = df.drop_duplicates(subset = ['Title', 'Artists'], keep = 'first')
df = df.reset_index(drop = True)

In [7]:
#get numerical features and apply standard scaler to them
numerical_features = df.columns[4:11]
scaler = StandardScaler()
df_scaled_numerical = scaler.fit_transform(df[numerical_features])

#Save scaler created for future predictions file
with open('scaler_numerical.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [8]:
#add in the scaled data to df and reset index
df.loc[:, numerical_features] = df_scaled_numerical
df = df.reset_index(drop = True)

In [9]:
#Track an artists previous average top rank of all their songs. We append the top rank of each song to the artist in dictionary
#and calculate the mean of the top ranks in the artists list to that point so we are still in respect to time
previous_song_average = {}
for index, row in df.iterrows():
    if row["Artists"] not in previous_song_average:
        previous_song_average[row["Artists"]] = [row["Highest Rank"]]
        df.at[index, "Previous Top Rank Avg"] = 201.0
    else:    
        df.at[index, "Previous Top Rank Avg"] = np.mean(previous_song_average[row["Artists"]])
        previous_song_average[row["Artists"]].append(row["Highest Rank"])
    

In [10]:
#drop first month
df = df[df['Date'] >= "2017-02-01"]

#rename df to the cleaned version
df_clean = df.copy()


In [11]:
df_clean.head(3)

,Rank,Title,Artists,Date,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,...,Points (Total),Points (Ind for each Artist/Nat),id,Song URL,Highest Rank,Previous Popularity,Number Songs Previous,Number of Artists,Top 10,Previous Top Rank Avg
300,197,I Miss You,"Grey, Bahari",2017-02-01,-0.166703,0.745431,0.196288,-0.310595,0.528030,-0.167878,...,4,2.0,1sl5tSzfK1JrcQIcA8rYDy,https://open.spotify.com/track/1sl5tSzfK1JrcQI...,175,0.0,0.0,2.0,False,201.0
301,188,Location,Khalid,2017-02-01,0.411958,-1.085786,-1.772413,2.446186,0.375981,-0.167878,...,13,13.0,152lZdxL1OR0ZMW6KquMif,https://open.spotify.com/track/152lZdxL1OR0ZMW...,33,0.0,0.0,1.0,False,201.0
302,199,Magic,"Thomas Gold, Jillian Edwards",2017-02-01,-0.308005,1.431411,0.192504,-0.718699,-0.744384,-0.155927,...,2,1.0,5abwHwUhywLGNli5NDcEN7,https://open.spotify.com/track/5abwHwUhywLGNli...,168,0.0,0.0,2.0,False,201.0


In [12]:
#save clean dataset to a csv file
df_clean.to_csv("top10_data.csv", index=False)